# Code Cloud

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1689759431511_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1689759431511_0001,pyspark,idle,Link,Link,✔


### 4.10.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### 4.10.3 Import des librairies

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [4]:
PATH = 's3://oc-ds-p8-fruits'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-ds-p8-fruits
PATH_Data:   s3://oc-ds-p8-fruits/Test
PATH_Result: s3://oc-ds-p8-fruits/Results

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://oc-ds-p8-fru...|2023-07-06 11:12:23|  7353|[FF D8 FF E0 00 1...|
|s3://oc-ds-p8-fru...|2023-07-06 11:12:24|  7350|[FF D8 FF E0 00 1...|
|s3://oc-ds-p8-fru...|2023-07-06 11:12:23|  7349|[FF D8 FF E0 00 1...|
|s3://oc-ds-p8-fru...|2023-07-06 11:12:24|  7348|[FF D8 FF E0 00 1...|
|s3://oc-ds-p8-fru...|2023-07-06 11:12:29|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------+----------+
|path                                              |label     |
+--------------------------------------------------+----------+
|s3://oc-ds-p8-fruits/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://oc-ds-p8-fruits/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://oc-ds-p8-fruits/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://oc-ds-p8-fruits/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://oc-ds-p8-fruits/Test/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------------+----------+
only showing top 5 rows

None

#### 4.10.5.2 Préparation du modèle

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 0s 0us/step

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [13]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [14]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://oc-ds-p8-fruits/Results

In [17]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [19]:
%pip install s3fs

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Install s3fs to access S3
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parquet.py", line 67, in _get_path_or_handle
    path_or_handle, **(storage_options or {})
  File "/usr/local/lib/python3.7/site-packages/fsspec/core.py", line 353, in url_to_fs
    chain = _un_chain(url, kwargs)
  File "/usr/local/lib/python3.7/site-packages/fsspec/core.py", line 315, in _un_chain
    cls = get_filesystem_class(protocol)
  File "/usr/local/lib/python3.7/site-packages/fsspec/registry.py", line 213, in get_filesystem_class
    raise ImportError(bit["err"]) from e
ImportError: Install s3fs to access S3



In [19]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                           path  ...                                           features
0    s3://p8-data/Test/Watermelon/r_174_100.jpg  ...  [0.0059991637, 0.44703647, 0.0, 0.0, 3.3713572...
1  s3://p8-data/Test/Pineapple Mini/128_100.jpg  ...  [0.0146466885, 4.080593, 0.055877004, 0.0, 0.0...
2  s3://p8-data/Test/Pineapple Mini/137_100.jpg  ...  [0.0, 4.9659867, 0.0, 0.0, 0.0, 0.0, 0.5144821...
3      s3://p8-data/Test/Watermelon/275_100.jpg  ...  [0.22511952, 0.07235509, 0.0, 0.0, 1.690149, 0...
4      s3://p8-data/Test/Watermelon/271_100.jpg  ...  [0.3286234, 0.18830013, 0.0, 0.0, 1.9123534, 0...

[5 rows x 3 columns]

In [20]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [21]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22688, 3)

# ACP Cloud Version

In [ ]:
%pip install s3fs

In [ ]:
df_source = pd.read_parquet(PATH_Result, engine='pyarrow')

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize an empty list to store the numpy arrays
numpy_arrays = [feature for feature in df_source['features'].values]
# Create a new DataFrame with the extracted numpy arrays
new_df = pd.DataFrame(numpy_arrays)

ss_df = pd.DataFrame(StandardScaler().fit_transform(new_df))
ss_df.head(2)

In [ ]:
new_df_spark = spark.createDataFrame(ss_df)

In [ ]:
from pyspark.ml.feature import VectorAssembler

colonne_name = [f"{i}" for i in range(new_df.shape[1])]
v_asmblr =  VectorAssembler(inputCols=colonne_name, outputCol='vectorized_features')
df = v_asmblr.transform(new_df_spark)

## 1. Fit PCA

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
k=9
pca = PCA(k=k, inputCol="vectorized_features") # k = 2 : on ne peut que les deux premieres composantes principales
pca.setOutputCol("pca_features")

In [ ]:
model = pca.fit(df)
model.getK()

In [ ]:
output_df = model.transform(df)

In [ ]:
print("Type output PCA transform : ", type(output_df))

In [ ]:
output_df.printSchema()

In [ ]:
pca_features = output_df.select('pca_features').collect()
print("Collected PCA_FEATURES caracteristiques")
print("----------------------------------------------")
print("Type : ", type(pca_features))
print("Taille : ", len(pca_features))
print("Type item[0] : ", type(pca_features[0]))

In [ ]:
pca_features[0]

In [ ]:
def transform_list_of_vect_to_df(vectlist) :
    return pd.DataFrame([[vectlist[j][0][i] for i in range(k)] for j in range(len(vectlist))])

In [ ]:
# On conserve les coordonnnees des points seulement pour le premier plan factoriel
pca_coord_pf1 = transform_list_of_vect_to_df(pca_features).iloc[:, :2]
pca_coord_pf1.head(2)

## 2. PCA analysis
### 2.1. Scree plot

In [ ]:
v_explained_variance = model.explainedVariance
v_explained_variance

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

x_list = range(1, k+1)
scree_values = v_explained_variance.toArray()
plt.figure(figsize=(10, 6))
plt.bar(x_list, scree_values)
plt.plot(x_list, np.cumsum(scree_values), color='r', marker='o')
plt.title(f"Scree plot ACP sur les {k} premieres composantes principales \n")
plt.show()

NameError: name 'k' is not defined

### 2.2. Projection sur le premier plan factoriel

#### Recuperation des noms de fruits
On va recuperer le nom des fruits pour colorer notre premier plan factoriel.

In [ ]:
def get_fruit_name(chemin, parent_folder='data') :
    dossier_images_parent = parent_folder
    nom_fruit = chemin.split(dossier_images_parent)[-1].split('/')[1] # Premier "/" est le debut de l'adresse
    cat_fruits = nom_fruit.split(' ')[0]
    return  nom_fruit, cat_fruits

In [ ]:
fruit_list = [get_fruit_name(chemin=chemin, parent_folder='Test')[0]  for chemin in df_source['path'].values]
cat_fruit_list = [get_fruit_name(chemin=chemin, parent_folder='Test')[1]  for chemin in df_source['path'].values]

In [ ]:
pca_coord_pf1['fruit'] = fruit_list
pca_coord_pf1['cat_fruit'] = cat_fruit_list
pca_coord_pf1.head(2)

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(data=pca_coord_pf1, x=0, y=1, hue='fruit', palette="rocket_r")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(data=pca_coord_pf1, x=0, y=1, hue='cat_fruit')
plt.show()